In [264]:
import random
import math
import numpy as np

import matplotlib.pyplot as plt

# Implementation


### Evolution Strategy

In [456]:
class evolution_strategy():
    
    def __init__(self, f, dim, low, high, mu=10, lam=70):  # mu = number of parents, lam = number of childs
        self.f = f
        self.dim = dim
        self.low = low
        self.high = high
        self.mu = mu
        self.lam = lam
        
    def _initialize_generation(self):
        """
        initialize generation on the zero iteration
        """
        self.current_generation = np.array([np.array((self.high - self.low) * \
                                             np.random.sample(self.dim) + \
                                             self.low) for i in range(self.mu)]) 

    def _add_selected_to_new_generation(self):
        """
        initialize new generation on the next iteration
        as selected from current generation
        """
        self.new_generation = self.current_generation    
    
    def _evaluate_fitness(self):
        """
        evaluate fitness-function of each element from current generation
        """
        self.current_fitness = np.array([self.f(el) for el in self.current_generation])
    
    # SELECTION STAGE
    def _run_selection(self):
        """
        select randomly two elements from elements with fitness-function > mean
        create current_parents as an array of two floats
        """
        self._evaluate_fitness()
        #print(self.current_fitness)
        indices = (-self.current_fitness.flatten()).argsort()[:self.mu]
        #print(indices)
        self.current_generation = np.array(self.current_generation)[indices] 
        if self.print_out:
            print('-> selected generation: ', self.current_generation)

    def _define_parents(self):
        """
        select randomly two elements from current_generation
                                        with fitness-function > mean
        create current_parents as an array of two floats
        """
        self._evaluate_fitness()
        m = self.current_fitness.mean()
        indices = np.where(self.current_fitness > m)[0]
        random.shuffle(indices)
        self.parents = np.array(self.current_generation)[indices[:2]] 
        if self.print_out:
            print('-> selected parents: ', self.parents)
            
        if self.new_generation == None:
            self.new_generation = self.parents
        else:
            for i in range(2):
                if not self.parents[i] in self.new_generation:
                    self.new_generation = np.vstack((self.new_generation, self.parents[i]))
        if self.print_out:
            print('-> new generation: ', self.new_generation)  
    
    # CROSSOVER STAGE
    def _make_child(self):
        """
        implement 1-point crossover
        create a child from two parents
        """
        point = np.random.randint(self.dim + 1)
        if self.print_out:
            print('-> crossover point: ', point)
            
        self.child = np.zeros((1,self.dim))
        if point != 0:
            self.child[:point] = self.parents[0][:point]
        if point != self.dim:
            self.child[point:] = self.parents[1][point:]
        
        if self.print_out:
            print('-> created child: ', self.child)      
        
    
    # MUTATION STAGE
    def _mutate(self):
        """
        inverse each bit of element with probability p
        :param: element - float
        return mutated element as float
        """
        if np.random.rand() < self.p_mut:
            mutated = self.child + (np.random.rand(dim) - 0.5) * 0.005
        else:
            mutated = self.child
        if self.print_out:
            print('->  mutated child: ', mutated)
        self.child = mutated
        
        if not self.child in self.new_generation:
            self.new_generation = np.vstack((self.new_generation, self.child))
        if self.print_out:
            print('-> new generation: ', self.new_generation)  
    
    # STOP CONDITIONS
    def _mean_gen_distance(self):
        d = 0
        n = len(self.current_generation)
        for i in range(n):
            for j in range(i,n):
                   d += np.linalg.norm(self.current_generation[i] - 
                                       self.current_generation[j])
        return d / n

    def _mean_fit_distance(self):
        d = 0
        n = len(self.current_fitness)
        for i in range(n):
            for j in range(i,n):
                   d += np.linalg.norm(self.current_fitness[i] -
                                       self.current_fitness[j])
        return d / n
    
    def _stop_condition(self, n_iter):
        return self._mean_gen_distance() < self.eps or \
               self._mean_fit_distance() < self.eps2 or \
               n_iter >= self.max_iter
    
    def run_algorithm(self, p_cross=0.5, p_mut=0.1,
                      eps=0.5, eps2=0.5, 
                      max_iter=10, print_out=True):
        self.p_cross = p_cross
        self.p_mut = p_mut
        self.eps = eps
        self.eps2 = eps2
        self.max_iter = max_iter
        self.print_out = print_out
        
        self._initialize_generation()  
        self._evaluate_fitness()
        n_iter = 0
        
        while not self._stop_condition(n_iter):
            #print(n_iter)
            try:
                if self.print_out:
                    print('-> current generation:', self.current_generation)
                self._run_selection()
                self.new_generation = None
                #self._add_selected_to_new_generation()
                for i in range(self.lam):
                    self._define_parents()
                    self._make_child()
                    self._mutate()
                self.current_generation = self.new_generation
                self._evaluate_fitness()
                if self.print_out:
                    print('-> new generation:', self.current_generation) # list(set(self.current_generation)))
                    print('-> fitness-function values:', self.current_fitness)
                n_iter += 1
            except:
                break
                
        return self.current_generation[self.current_fitness.argmax()], self.current_fitness.max()

In [457]:
f = lambda x: x / ((1 + x ** 2) ** 2)
dim = 1
low = -2
high = 2
ES = evolution_strategy(f, dim, low, high)
x0, f0 = ES.run_algorithm(max_iter=500, print_out=False, eps=1e-4, eps2=1e-4)
print('x0 = ', x0)
print('f0 = ', f0)

/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


x0 =  [ 0.59886292]
f0 =  0.324429754777


In [458]:
from scipy.optimize import minimize
print('x0 = ', minimize(lambda x: -f(x), np.zeros((1))).x)
print('f0 = ', minimize(lambda x: -f(x), np.zeros((1))).fun * (-1))

x0 =  [ 0.57735025]
f0 =  0.3247595264191642


In [459]:
f = lambda x: 2 * x[0] - x[1]**3 + (1 + x[0])**.5  + 10
dim = 2
low = 0
high = 15
ES = evolution_strategy(f, dim, low, high)
x0, f0 = ES.run_algorithm(max_iter=500, print_out=False, eps=1e-4, eps2=1e-4)
print('x0 = ', x0)
print('f0 = ', f0)

/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


x0 =  [ 12.10348996   2.0483176 ]
f0 =  29.2329226679


In [460]:
from scipy.optimize import minimize
print('x0 = ', minimize(lambda x: -f(x), np.zeros((dim))).x)
print('f0 = ', minimize(lambda x: -f(x), np.zeros((dim))).fun * (-1))

x0 =  [  9.48300787e+08   0.00000000e+00]
f0 =  1896632378.2014613


In [367]:
?minimize